In [29]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
from matplotlib import pyplot as plt

In [2]:
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))
from src.config import DATA_DIR

In [3]:
path1 = DATA_DIR / "portfolio" / "J6_K6_skip0_returns.csv"
if not path1.exists():
        raise FileNotFoundError(f"Portfolio data not found at {path1}. Run portfolio.py first.")
else:
    portfolio = pd.read_csv(
        path1, 
        parse_dates=["Date"]
        ).set_index("Date")["spread"]

In [4]:
train_end = '2018-01-01'
returns = portfolio.dropna().copy()
ret_train = returns.loc[:train_end]
ret_test = returns.loc[train_end:]

In [5]:
crash_q = ret_train.quantile(0.10)
print("Crash threshold:", crash_q)

Crash threshold: -0.03887094537462145


In [16]:
crash_label = (returns.shift(-1)<=crash_q).astype(int)
crash_label = crash_label.rename("crash_label")

In [7]:
from src.risk_metrics import cumulative_returns, drawdown_series
from src.betas import compute_rolling_beta

In [8]:
path2 = DATA_DIR / "portfolio" / "market_returns.csv"
if not path2.exists():
    raise FileNotFoundError(f"Risk-free rate data not found at {path2}. Run capm_params.py first.")
else:
    market = pd.read_csv(
        path2,
        parse_dates=["Date"]
        ).set_index("Date")["Market_Returns"]

In [9]:
features = pd.DataFrame(index=returns.index)
features['1m_ret'] = returns
features['3m_ret'] = returns.rolling(3).mean()
features['6m_ret'] = returns.rolling(6).mean()

features['3_vol'] = returns.rolling(3).std()
features['6_vol'] = returns.rolling(6).std()

drawdown = drawdown_series(cumulative_returns(returns))
features['drawdown'] = drawdown.reindex(features.index)

features['market_ret'] = market.reindex(features.index)
features['3m_market_ret'] = market.rolling(3).mean().reindex(features.index)

betas = compute_rolling_beta(returns, market, 6)
features['beta'] = betas.reindex(features.index)

In [11]:
features.head(10)

,1m_ret,3m_ret,6m_ret,3_vol,6_vol,drawdown,market_ret,3m_market_ret,beta
Date,,,,,,,,,
2001-02-01,0.212342,NaN,NaN,NaN,NaN,0.000000,-0.092291,-0.017867,NaN
2001-03-01,-0.007522,NaN,NaN,NaN,NaN,-0.007522,-0.064205,-0.040620,NaN
2001-04-01,-0.139201,0.021873,NaN,0.177605,NaN,-0.145676,0.076814,-0.026560,NaN
2001-05-01,0.048643,-0.032693,NaN,0.096419,NaN,-0.104119,0.005090,0.005900,NaN
2001-06-01,-0.018337,-0.036298,NaN,0.095202,NaN,-0.120547,-0.025035,0.018956,NaN
2001-07-01,0.031072,0.020459,0.021166,0.034728,0.114457,-0.093220,-0.010740,-0.010228,-1.568331
2001-08-01,0.070247,0.027661,-0.002516,0.044390,0.074831,-0.029522,-0.064108,-0.033295,-1.056050
2001-09-01,0.054176,0.051832,0.007767,0.019692,0.078170,0.000000,-0.081723,-0.052191,-1.169359
2001-10-01,0.031251,0.051891,0.036175,0.019598,0.030536,0.000000,0.018099,-0.042578,-0.293642


In [17]:
data = features.join(crash_label,how='inner').dropna()
data.head(10)

,1m_ret,3m_ret,6m_ret,3_vol,6_vol,drawdown,market_ret,3m_market_ret,beta,crash_label
Date,,,,,,,,,,
2001-07-01,0.031072,0.020459,0.021166,0.034728,0.114457,-0.093220,-0.010740,-0.010228,-1.568331,0
2001-08-01,0.070247,0.027661,-0.002516,0.044390,0.074831,-0.029522,-0.064108,-0.033295,-1.056050,0
2001-09-01,0.054176,0.051832,0.007767,0.019692,0.078170,0.000000,-0.081723,-0.052191,-1.169359,0
2001-10-01,0.031251,0.051891,0.036175,0.019598,0.030536,0.000000,0.018099,-0.042578,-0.293642,1
2001-11-01,-0.100181,-0.004918,0.011371,0.083293,0.062303,-0.100181,0.075176,0.003851,-0.911002,0
2001-12-01,-0.006513,-0.025148,0.013342,0.067669,0.061355,-0.106042,0.007574,0.033616,-0.946007,0
2002-01-01,0.065172,-0.013841,0.019025,0.082920,0.064808,-0.047781,-0.015574,0.022392,-0.960646,0
2002-02-01,0.016587,0.025082,0.010082,0.036589,0.059838,-0.031986,-0.020766,-0.009589,-0.946686,1
2002-03-01,-0.090502,-0.002914,-0.014031,0.079648,0.067212,-0.119593,0.036739,0.000133,-1.618343,0


In [18]:
X = data.drop(columns = "crash_label")
y = data["crash_label"]

In [19]:
X_train = X.loc[:train_end]
X_test = X.loc[train_end:]
y_train = y.loc[:train_end]
y_test = y.loc[train_end:]

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [22]:
clf_model = Pipeline([
    ("scaler", StandardScaler()),
    ("logit", LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=1000
    ))
])

clf_model.fit(X_train, y_train)

/Users/vaishnukanna/ml/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('logit', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'l2'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with 

In [23]:
prob_crash = pd.Series(clf_model.predict_proba(X_test)[:,1], index=X_test.index, name="prob_crash")

In [24]:
exposure = 1-prob_crash
exposure.name = "exposure"
exposure.describe()

count    72.000000
mean      0.901277
std       0.074134
min       0.489187
25%       0.883869
50%       0.921846
75%       0.940007
max       0.978843
Name: exposure, dtype: float64

In [43]:
test = returns.loc[prob_crash.index]
returns_filtered = test*exposure
returns_filtered.name = "mom_returns_filtered"
returns_baseline = test.rename("mom_returns_baseline")

In [44]:
print(returns_baseline.describe())
print(returns_filtered.describe())

count    72.000000
mean      0.018863
std       0.052253
min      -0.142118
25%      -0.007757
50%       0.017983
75%       0.052475
max       0.162559
Name: mom_returns_baseline, dtype: float64
count    72.000000
mean      0.017293
std       0.044437
min      -0.119310
25%      -0.007216
50%       0.016743
75%       0.047514
max       0.129014
Name: mom_returns_filtered, dtype: float64


In [45]:
print(pd.concat([returns_baseline, returns_filtered, exposure], axis=1).head())
print(X_test.shape)
print(X_train.shape)
print(prob_crash.describe())

            mom_returns_baseline  mom_returns_filtered  exposure
Date                                                            
2018-01-01              0.052365              0.050147  0.957649
2018-02-01              0.067944              0.063006  0.927313
2018-03-01              0.016391              0.014722  0.898203
2018-04-01              0.009184              0.008060  0.877577
2018-05-01              0.075028              0.068709  0.915784
(72, 9)
(199, 9)
count    72.000000
mean      0.098723
std       0.074134
min       0.021157
25%       0.059993
50%       0.078154
75%       0.116131
max       0.510813
Name: prob_crash, dtype: float64


In [46]:
from src.risk_metrics import sharpe_ratio, max_drawdown, annualized_volatility, skewness

In [47]:
path3 = DATA_DIR / "portfolio" /"rf_rate.csv"
if not path2.exists():
    raise FileNotFoundError(f"Risk-free rate data not found at {path3}. Run capm_params.py first.")
else:
    rf = pd.read_csv(
        path3,
        parse_dates=["Date"]
    ).set_index("Date")["Rf_Rate"]

In [49]:
compare = {
    "Baseline model": {
        "Annualized Volatility": annualized_volatility(returns_baseline),
        "Sharpe": sharpe_ratio(returns_baseline, rf),
        "Max Drawdown": max_drawdown(returns_baseline),
        "Skew": skewness(returns_baseline)
    },
    "After adjusting Exposure based on carsh probability": {
        "Annualized Volatility": annualized_volatility(returns_filtered),
        "Sharpe": sharpe_ratio(returns_filtered, rf),
        "Max Drawdown": max_drawdown(returns_filtered),
        "Skew": skewness(returns_filtered)
    },
}

risk_table = pd.DataFrame(compare).T
risk_table

,Annualized Volatility,Sharpe,Max Drawdown,Skew
Baseline model,0.181009,0.095759,-0.142118,-0.229451
After adjusting Exposure based on carsh probability,0.153935,0.102394,-0.119310,-0.284988


- Volatility is reduced by ~15% (from 18.10% to 15.39%), indicating that the crash-risk classifier successfully lowers exposure during high-risk momentum regimes.

- Maximum drawdown improves by ~16% (from −14.21% to −11.93%), demonstrating meaningful downside protection without eliminating exposure to the momentum premium.

- Risk-adjusted performance improves modestly, with the Sharpe ratio increasing from 0.096 to 0.102, while skewness becomes slightly more negative due to the linear exposure-scaling mechanism.